In [2]:
import os
os.chdir('..') # this resolves ImportError: attempted relative import with no known parent package
# import sys
# parent_dir = os.getcwd()
# sys.path.append(parent_dir)
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from src.exploration.data_manipulation import single_metric_dataframes, split_into_video_series

In [3]:
raw_data = pd.read_csv("data/processed/json_data_processed.csv")
data = raw_data.copy()
df = pd.DataFrame(data)
df.head()

,videoID,publishedAt,channelId,title,description,channelTitle,tags,viewCount,likeCount,favoriteCount,commentCount,duration_timedelta,duration_hhmmss
0,uGtc9Bu9Txk,2020-12-31T23:00:10Z,UCArk93C2pbOvkv6jWz-3kAg,🍾 FORD KIERNAN HOGMANAY SPECIAL! | Open Goal,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'still game', 'jack ...",205682,2286,0,112,0 days 01:21:55,01:21:55
1,vAGmV-mRRT0,2020-12-24T07:00:08Z,UCArk93C2pbOvkv6jWz-3kAg,CHRISTMAS EVE SPECIAL | Right in the Coupon w/...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'betting', 'slaney',...",63357,742,0,30,0 days 00:44:31,00:44:31
2,m54LPZSlStQ,2020-12-22T17:00:20Z,UCArk93C2pbOvkv6jWz-3kAg,EPIC CUP FINAL REVIEW | Keeping the Ball on th...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'celtic vs hearts', ...",85262,976,0,63,0 days 01:06:12,01:06:12
3,1RcFj1Sfs58,2020-12-21T17:00:03Z,UCArk93C2pbOvkv6jWz-3kAg,JAMIE CARRAGHER | Open Goal Meets...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'Jamie Carragher', '...",855101,4966,0,661,0 days 02:22:14,02:22:14
4,MaJhyEr6csQ,2020-12-17T17:25:28Z,UCArk93C2pbOvkv6jWz-3kAg,CELTIC v HEARTS & ST MIRREN END RANGERS UNBEAT...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'celtic v hearts', '...",64383,659,0,49,0 days 00:45:40,00:45:40


In [4]:
# convert publishedAt values from string to datetime
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
# use strftime to format the datetime value into just year and month so we can group by year and month
df['publishedAt_year_month'] = df['publishedAt'].dt.strftime("%Y-%m")

# group the views, likes and comment numbers by month
df_group = (
    df.groupby(by=[df['publishedAt_year_month']])[
    ['viewCount', 'likeCount', 'commentCount']
    ]
    .sum()
    .astype(int)
    )

df_group.reset_index(inplace=True)
df_group.head(10)

,publishedAt_year_month,viewCount,likeCount,commentCount
0,2017-05,8709,37,2
1,2017-06,767417,2577,87
2,2017-07,372375,1610,64
3,2017-08,373218,1793,80
4,2017-09,172559,1152,37
5,2017-10,438256,2429,134
6,2017-11,210776,1366,82
7,2017-12,400433,2658,105
8,2018-01,149222,1296,93
9,2018-02,187810,1326,71


# Dash app

Dash Bootstrap is a powerful tool for styling Dash apps, helping us cre- ate the layout, style the app, and add Bootstrap components such as buttons and radio items.

### \__name__
The __name__ parameter serves as the name of the Dash application. It's a special variable that holds the name of the module or script where the code is executed. When you run the script directly, __name__ will be set to "__main__". However, if you import the module containing the Dash app into another script or application, __name__ will be the name of the imported module.


Using __name__ as the name of the Dash app ensures that each instance of the app has a unique identifier, which is crucial for avoiding name conflicts when running multiple Dash apps simultaneously or in conjunction with other components. It also allows for more modular code, as the app can be defined and reused in different contexts without requiring explicit naming.

In summary, the __name__ parameter plays a vital role in identifying and distinguishing Dash applications, promoting modularity and enabling seamless integration within various Python projects.

In [6]:
# choose theme and assign it to the external stylesheets parameter
app = Dash(name=__name__, external_stylesheets=[dbc.themes.BOOTSTRAP]) # available themes: https://hellodash.pythonanywhere.com (click change theme button)

##### layout of the app #####
# We generally refer to the layout of an app as a grid, which commonly consists of 12 columns and an infinite number of rows
# To start building the layout, we need to create a container that will house all our rows and columns as well as the components that 
# will go inside them. The dbc.Container syntax is pretty much like an html.Div, but it is more compatible with Bootstrap styling. 
# First we’ll declare the rows, then we’ll declare the columns that go inside each row. 
# Finally, we’ll put the app components inside the columns. This final step defines the location of each component on the page.



# start building - start with general layout p.79 book of dash

###### NOTE - STILL NEED TO ADD ALL THE PARAMETERS FOR EACH HTML, DCC, AND DBC COMPONENTS ######
app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                [
                    html.H1(title="Test"),
                    dcc.Graph()
                ],
                width=12,
            )
        ),

        dbc.Row(
            dbc.Col(
                [
                    dbc.Label(),
                    dbc.RadioItems(),
                ],
                width=4,
            )   
        ),
        dbc.Row(
            [
            dbc.Col(
                [
                    dbc.Label(),
                    dcc.RangeSlider(),
                    dbc.Button(),   
                ],
                width=6,
            ),
            ]
        ),
    ]
)

